Generate random cores with any size

In [1]:
NUM_GAMES = 2
from typing import Callable
import random
from haskellian import iter as I, promise as P, either as E
import pure_cv as vc
from moveread.core import CoreAPI, Game, Image
from moveread.annotations.images import ImageMeta
from moveread.sdk import MovereadAPI
api = MovereadAPI.at('/home/m4rs/.core/llobregat23/')

In [2]:
def choose_image(images: list[Image]) -> str:
  for img in images:
    meta = ImageMeta.model_validate(img.meta)
    if meta.source == 'camera' or meta.source == 'corrected-scan':
      return img.url
  return images[0].url

async def read_game(
  gameId: str, core: CoreAPI,
  choose_image: Callable[[list[Image]], str] = choose_image
) -> tuple[Game, list[list[bytes]]]:
  game = (await core.games.read(gameId)).unsafe()
  imgs = await P.all([
    P.all([core.blobs.read(choose_image(sheet.images)).then(E.unsafe) for sheet in player.sheets])
    for player in game.players
  ])
  return game, imgs

async def sample_games(source: MovereadAPI, n: int) -> list[str]:
  gameIds = E.sequence(await source.games.list()).unsafe()
  indices = random.sample(range(len(gameIds)), n)
  return I.pick(indices, gameIds)

async def sample_core(src: MovereadAPI, dst: MovereadAPI, num_games: int):
  gameIds = await sample_games(src, num_games)
  for i, gid in enumerate(gameIds):
    print(f'\r{i+1} / {len(gameIds)} [{gid}]\t', end='', flush=True)
    _, imgs = await read_game(gid, api.core)
    await dst.games.create(gid, imgs)
  print()

In [3]:
for n in [2, 5, 10, 25, 50, 100]:
  print(n)
  output = f'/home/m4rs/.core/dev-cores/{n}'
  dst = MovereadAPI.debug(output)
  await sample_core(api, dst, n)

2
2 / 2 [b/6/3]		
5
5 / 5 [a/6/45]	
10
10 / 10 [b/5/23]	
25
25 / 25 [a/5/41]	
50
50 / 50 [a/5/101]	
100
100 / 100 [a/2/1]		
